In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-21 07:38:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-21 07:38:35 (6.01 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData_Amazon_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

amazon_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"

# Dataset 1 --> Video Games
dataset1 = "amazon_reviews_us_Video_Games_v1_00.tsv.gz"

amazon_url1 = amazon_url + dataset1
spark.sparkContext.addFile(amazon_url1)
video_games_df = spark.read.csv(SparkFiles.get(dataset1), inferSchema=True, sep="\t", header=True)

# Show DataFrame
video_games_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                                              |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                     

In [ ]:
# Count the number of records(rows) in the complete dataset.
print('Number of records in Video games dataset:', video_games_df.count())

Number of records in Video games dataset: 1785997


In [ ]:
# Transform the dataset to fit the table review_id_table
from pyspark.sql.types import DateType

review_id_df = video_games_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_id_table = review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))

print('Number of rows in review_id_table:' + str(review_id_table.count()))
review_id_table.orderBy('review_id').show(truncate=False)

Number of rows in review_id_table:1785997
+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10003I619LWL0|4317219    |B001REZLY8|816407837     |2015-01-10 |
|R100078OO83YQB|31850030   |B0016HM45K|670563392     |2008-12-29 |
|R10009XH2FY9Q0|51098385   |B00000IOQV|845394808     |1999-11-28 |
|R1000EJULTHQ16|12211018   |B002I0H7K6|22716581      |2012-02-05 |
|R1000GECIM9DZG|10430321   |B006SVUQNQ|523228689     |2014-10-06 |
|R1000ID99V4TKO|4971033    |B00KY1HZ80|886096191     |2015-03-18 |
|R1000QK6J2WSFR|15127738   |B0018K77RA|139949924     |2011-01-10 |
|R1000RERQGPWNO|42546325   |B002S5NKIY|469043465     |2010-02-17 |
|R1000U9GM6NPGS|13626862   |B00DUJKIO6|518264079     |2014-01-23 |
|R1000WZPINB01L|43132564   |B0041HROGG|166509481     |2012-11-07 |
|R1000XAVJH7WUV|25431956   |B000087L4G|947390207     |2005-06-09 |
|R1000YD5XB0Q9H|4549

In [ ]:
# 'review_id_df' Column list and its data types
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Transform the dataset to fit the table 'products' and table count
products_df = video_games_df.select('product_id', 'product_title').distinct()
print("Distinct count of products:", products_df.count())
products_df.orderBy('product_id').show(truncate=False)

Distinct count of products: 65792
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                               |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0000118532|Bethesda The Evil Within Xbox One                                                                                                                                           |
|006056038X|Bitter Truth                                                                                                                                                                |
|006073132X|Freakonomics: A Rogue Ec

In [ ]:
# 'products_df' Column list and its data types
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Transform the dataset to fit the 'customer' table and print the count of records
customersCounts = video_games_df.select('customer_id').groupBy('customer_id').count()
customers_df = customersCounts.withColumnRenamed('count', 'customer_count')

print('Number of records in customers: '+ str(customers_df.count()))
customers_df.orderBy('customer_id').show(truncate=False)

Number of records in customers: 1045733
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|10018      |4             |
|10114      |2             |
|10146      |1             |
|10164      |1             |
|10192      |1             |
|10206      |8             |
|10267      |1             |
|10323      |1             |
|10382      |1             |
|10453      |1             |
|10467      |1             |
|10481      |1             |
|10605      |1             |
|10615      |1             |
|10664      |1             |
|10670      |1             |
|10685      |1             |
|10704      |1             |
|10776      |1             |
|10866      |1             |
+-----------+--------------+
only showing top 20 rows



In [ ]:
# 'customers' Table column list and its data types
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# Transform the dataset to fit 'vine_table'
vine_table= video_games_df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
print('Count of records in Vine dataset:' + str(vine_table.count()))
print('Count of records in Vine dataset:' + str(vine_table.distinct().count()))
vine_table.orderBy('review_id').show(truncate=False)

Count of records in Vine dataset:1785997
Count of records in Vine dataset:1785997
+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10003I619LWL0|5          |1            |1          |N   |
|R100078OO83YQB|5          |0            |0          |N   |
|R10009XH2FY9Q0|5          |4            |4          |N   |
|R1000EJULTHQ16|5          |0            |0          |N   |
|R1000GECIM9DZG|5          |0            |0          |N   |
|R1000ID99V4TKO|5          |0            |0          |N   |
|R1000QK6J2WSFR|5          |0            |0          |N   |
|R1000RERQGPWNO|4          |0            |0          |N   |
|R1000U9GM6NPGS|5          |0            |0          |N   |
|R1000WZPINB01L|4          |0            |0          |N   |
|R1000XAVJH7WUV|3          |6            |9          |N   |
|R1000YD5XB0Q9H|3          |0            |1          |N   |
|R1000YRB7RKBIC|1 

In [ ]:
# 'vine_table' column list and its data types
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
#Create the Database properties

mode = "append"
db_url = "jdbc:postgresql://<AWS RDS endpoint>:5432/postgres"

config = {
    "user" : "<username>", 
    "password" : "<password>",
    "driver" : "org.postgresql.Driver"
}

In [ ]:
# Load 'review_id_df' dataframe into 'review_id_table

review_id_df.write.jdbc(url=db_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Load dataframe 'products_df' into RDS Table 'products'

products_df.write.jdbc(url=db_url, table='products', mode=mode, properties=config)

In [ ]:
# Load dataframe 'customers_df' into RDS Table 'customers'

customers_df.write.jdbc(url=db_url, table='customers', mode=mode, properties=config)

In [ ]:
# Load dataframe 'vine_table' into RDS Table 'vine_table'

vine_table.write.jdbc(url=db_url, table='vine_table', mode=mode, properties=config)